In [1]:

import pandas as pd
from multiprocessing import Pool
from nltk.corpus import stopwords
from nltk.corpus import stopwords
import numpy as np
stopWords = []
for i in """!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'""":
    stopWords.append(i)
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from operator import itemgetter
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from typing import List, Dict
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale

import numpy as np
np.random.seed(42)
import pandas as pd
from multiprocessing import Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '32'


def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word

from nltk.corpus import stopwords
stopWords = []
for i in """!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'""":
    stopWords.append(i)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
lemma_dict = {}
def lemmatizer(word):
    try:
        word = word.lower()
        if word in lemma_dict:
            return lemma_dict[word]
        else:
            normal_form = ps.stem(word)
            lemma_dict[word] = normal_form
            return normal_form
    except:
        return 'nonascii'
def join_2(x,*args):
    if len(args) == 0:
        return x
    else:
        return x+" " +args[0]
def transform(x):
    try:
        for symbol in stopWords:
            if symbol in x:
                x = x.replace(symbol," ")
                x = x.replace('  '," ")
                x = x.replace('   '," ")
        x = x.split()
        x = map(lemmatizer,x)
        #x = map(asc,x)
        x =  reduce(join_2,x)
        return x
    except:
        return "problem"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y = train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
train.drop(['toxic','severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

merge = pd.concat((train,test))

ntrain = train.shape[0]
print(train.shape,test.shape,merge.shape)

def how_many_upper(text):
    upper = 0
    for char in text:
        if char.isupper():
            upper+=1
    return upper

def text_len(text):
    return len(text)

def count_of_stop_words(text):
    count = 0
    for char in text:
        if char in stopWords:
            count+=1
    return count

def clean_text(text):
    for symbol in stopWords:
        text = text.replace(symbol," ")
        text = text.lower()
        text = text.replace('  '," ")
        text = text.replace('   '," ")
    return text

pictures = ['.gif','.jpg','.png','.tiff']

def has_picture(text):
    has = 0
    for _ in pictures:
        if _ in text:
            has = 1
    return has

(159571, 2) (153164, 2) (312735, 2)


In [3]:
p = Pool(processes=32)
merge['upper_symbols'] = p.map(how_many_upper, merge.comment_text.values)
p.terminate()

p = Pool(processes=32)
merge['text_len'] = p.map(text_len, merge.comment_text.values)
p.terminate()

merge['percentage_of_upper'] = merge.upper_symbols/merge.text_len

p = Pool(processes=32)
merge['stop_words_count'] = p.map(count_of_stop_words, merge.comment_text.values)
p.terminate()

merge['percentage_of_stop'] = merge.stop_words_count/merge.text_len

p = Pool(processes=32)
merge['has_picture'] = p.map(has_picture, merge.comment_text.values)
p.terminate()

p = Pool(processes=32)
merge.comment_text = p.map(transform, merge.comment_text.values)
p.terminate()

def split(text):
    return len(text.split())

p = Pool(processes=32)
merge['number_of_words'] = p.map(split, merge.comment_text.values)
p.terminate()

words_95 = np.percentile(merge.number_of_words,95)


merge.text_len = scale(merge.text_len)
merge.upper_symbols = scale(merge.upper_symbols)
merge.stop_words_count = scale(merge.stop_words_count)
merge.number_of_words = scale(merge.number_of_words)

In [4]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [5]:
folds = list(range(len(train)))
folds = np.array(folds) % 4

In [6]:
def lemma(text):
    return " ".join(lemmatize_all(str(text)))
train_holder = train['comment_text'].copy() 
test_holder = test['comment_text'].copy() 
from multiprocessing import Pool


In [7]:

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

X_train1=[]
X_test1=[]
#Function call to lemmatize X_train and X_test
print ("Train data lemmatization begins")

    
p = Pool(processes=32)
train_holder = p.map(lemma, train.comment_text.values)
p.terminate()
X_train1 = list(train_holder)
    
print ("Train data lemmatization ends")
print ("Test data lemmatization begins")

    
p = Pool(processes=32)
test_holder = p.map(lemma, test.comment_text.values)
p.terminate()
X_test1 = list(test_holder)    

print ("Test data lemmatization ends")

max_features = 50000
maxlen = 150
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train1) + list(X_test1))
X_train = tokenizer.texts_to_sequences(X_train1)
X_test = tokenizer.texts_to_sequences(X_test1)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)


def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))



Train data lemmatization begins
Train data lemmatization ends
Test data lemmatization begins
Test data lemmatization ends


In [8]:
from keras.layers import Input, Dropout, Dense, BatchNormalization,MaxPooling1D, \
    Activation, concatenate, GRU, Embedding, Flatten,LSTM
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping#, TensorBoard
from keras import backend as K
from keras import optimizers
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Conv1D

In [16]:
def get_model():
    inp = Input(shape=(maxlen, ), name="X_train")
    
    upper_symbols = Input(shape=[1], name="upper_symbols")
    text_len = Input(shape=[1], name="text_len")
    percentage_of_upper = Input(shape=[1], name="percentage_of_upper")
    stop_words_count = Input(shape=[1], 
                          name="stop_words_count")
    percentage_of_stop = Input(shape=[1], name="percentage_of_stop")
    has_picture = Input(shape=[1], name="has_picture")
    number_of_words = Input(shape=[1], name="number_of_words")
    
    x_1 = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x_1)
    
    x = Conv1D(64,3,activation='relu')(x)
    x = Conv1D(64,3,activation='relu')(x)  
    x = BatchNormalization()(x)
    
    x = Conv1D(128,3,activation='relu')(x)
    x = Conv1D(128,3,activation='relu')(x)  
    x = BatchNormalization()(x)
    
    x = Conv1D(256,3,activation='relu')(x)
    x = Conv1D(256,3,activation='relu')(x)  
    x = BatchNormalization()(x)
    
    x = Conv1D(512,3,activation='relu')(x)
    x = Conv1D(512,3,activation='relu')(x)  
    x = BatchNormalization()(x)
    
    x = Bidirectional(GRU(128, return_sequences=True,activation='relu', dropout=0.3, recurrent_dropout=0.))(x)
    x = Bidirectional(GRU(128, return_sequences=True,activation='relu', dropout=0.3, recurrent_dropout=0.))(x)
    
    y = Bidirectional(GRU(128, return_sequences=True,activation='relu', dropout=0.3, recurrent_dropout=0.))(x_1)
    y = Bidirectional(GRU(128, return_sequences=False,activation='relu', dropout=0.3, recurrent_dropout=0.))(y)
    
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool,y,
        upper_symbols
        , text_len
        , percentage_of_upper
        , stop_words_count
        , percentage_of_stop
        , has_picture 
        , number_of_words])
    
    conc = Dense(128,activation='relu')(conc)
    conc = Dropout(0.1)(conc)
    conc = Dense(128,activation='relu')(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=[inp, upper_symbols,text_len,percentage_of_upper,stop_words_count,percentage_of_stop,
                  has_picture,number_of_words], outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model



In [17]:
def make_dict_predict(df_1,df_2):
    dict_X_train = {'X_train' : df_1 ,'upper_symbols' : df_2.upper_symbols,
              'text_len' : df_2.text_len,'percentage_of_upper' : df_2.percentage_of_upper,
    'stop_words_count':df_2.stop_words_count,'percentage_of_stop':df_2.percentage_of_stop,
                'has_picture':df_2.has_picture,'number_of_words': df_2.number_of_words}
    return dict_X_train

def make_dict_train(df_1,df_2,fold):
    dict_X_train = {'X_train' : df_1[folds != fold] ,'upper_symbols' : df_2.upper_symbols[folds != fold],
              'text_len' : df_2.text_len[folds != fold],'percentage_of_upper' : df_2.percentage_of_upper[folds != fold],
    'stop_words_count':df_2.stop_words_count[folds != fold],'percentage_of_stop':df_2.percentage_of_stop[folds != fold],
                'has_picture':df_2.has_picture[folds != fold],'number_of_words': df_2.number_of_words[folds != fold]}
    return dict_X_train

def make_dict_OOF(df_1,df_2,fold):
    dict_X_train = {'X_train' : df_1[folds == fold] ,'upper_symbols' : df_2.upper_symbols[folds == fold],
              'text_len' : df_2.text_len[folds == fold],'percentage_of_upper' : df_2.percentage_of_upper[folds == fold],
    'stop_words_count':df_2.stop_words_count[folds == fold],'percentage_of_stop':df_2.percentage_of_stop[folds == fold],
                'has_picture':df_2.has_picture[folds == fold],'number_of_words': df_2.number_of_words[folds == fold]}
    
    return dict_X_train

In [ ]:
dict_OOF_predict = {}
dict_y_predict = {}

for fold in range(4):
    model = get_model()
    
    callbacks = [EarlyStopping(monitor='val_loss',
                               patience=4,
                               verbose=1,
                               min_delta=1e-4),
                 ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.1,
                                   patience=2,
                                   cooldown=2,
                                   verbose=1),
                 ModelCheckpoint(filepath="fold_"+str(fold)+"pooled_GRU_feat.hdf5",
                                 save_best_only=True,
                                 save_weights_only=True,verbose=1,monitor='val_loss')]


    dict_X_train = make_dict_train(x_train,merge[:ntrain],fold)
    dict_X_train_OOF = make_dict_OOF(x_train,merge[:ntrain],fold)
    dict_y = make_dict_predict(x_test,merge[ntrain:])

    
    model.fit(dict_X_train,y_train[folds != fold],epochs=200,
              validation_data=(dict_X_train_OOF,y_train[folds == fold]),callbacks=callbacks,batch_size=256)
    model.load_weights("fold_"+str(fold)+"pooled_GRU_feat.hdf5")
    
    predicts = model.predict(dict_y)
    dict_y_predict["fold_"+str(fold)] = predicts

    predicts_OOF = model.predict(dict_X_train_OOF)
    dict_OOF_predict["fold_"+str(fold)] = predicts_OOF
    
    del model
    print('fold {} is finished'.format(fold))
    

Train on 119678 samples, validate on 39893 samples
Epoch 1/200
119552/119678 [============================>.] - ETA: 4s - loss: 0.0670 - acc: 0.9772 
Epoch 00001: val_loss improved from inf to 0.05623, saving model to fold_0pooled_GRU_feat.hdf5
119678/119678 [==============================] - 4882s 41ms/step - loss: 0.0670 - acc: 0.9772 - val_loss: 0.0562 - val_acc: 0.9795
Epoch 2/200
119552/119678 [============================>.] - ETA: 4s - loss: 0.0492 - acc: 0.9817 
Epoch 00002: val_loss improved from 0.05623 to 0.05536, saving model to fold_0pooled_GRU_feat.hdf5
119678/119678 [==============================] - 4508s 38ms/step - loss: 0.0492 - acc: 0.9817 - val_loss: 0.0554 - val_acc: 0.9809
Epoch 3/200
119552/119678 [============================>.] - ETA: 4s - loss: 0.0460 - acc: 0.9826 
Epoch 00003: val_loss improved from 0.05536 to 0.04334, saving model to fold_0pooled_GRU_feat.hdf5
119678/119678 [==============================] - 4180s 35ms/step - loss: 0.0460 - acc: 0.9826 - v

In [12]:
f_0,f_1,f_2,f_3 = dict_y_predict.keys()
sub = dict_y_predict[f_0] + dict_y_predict[f_1] + dict_y_predict[f_2] + dict_y_predict[f_3]
sub/= 4.0

In [13]:
f_0 = 'fold_0'
f_1 = 'fold_1'
f_2 = 'fold_2'
f_3 = 'fold_3'
OOF_X = np.zeros_like(y_train)
OOF_X = pd.DataFrame(OOF_X)

OOF_X[folds==0] = dict_OOF_predict[f_0]
OOF_X[folds==1] = dict_OOF_predict[f_1]
OOF_X[folds==2] = dict_OOF_predict[f_2]
OOF_X[folds==3] = dict_OOF_predict[f_3]

pd.DataFrame(OOF_X).to_csv('OOF_toxic_pooled_GRU_50000_add_features_conv.csv',index=None)

In [14]:
submit = pd.read_csv('sample_submission.csv')
submit[['toxic','severe_toxic','obscene','threat','insult','identity_hate']] = sub
submit.to_csv('toxic_pooled_GRU_50000_add_features_conv.csv',index=None)

pd.DataFrame(OOF_X).to_csv('second_level/OOF_toxic_pooled_GRU_50000_add_features_conv.csv',index=None)
submit.to_csv('second_level/toxic_pooled_GRU_50000_add_features_conv.csv',index=None)

In [15]:
from sklearn.metrics import roc_auc_score,accuracy_score
print(roc_auc_score(np.array(y_train),(np.array(OOF_X))))

0.986704299277824


In [ ]:
9844